### Setup and Imports

This cell installs the necessary libraries and imports them for the RAG pipeline.

In [1]:
# Install required packages
!pip install -q unsloth transformers bitsandbytes sentence-transformers python-docx nltk scikit-learn datasets torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Import necessary libraries
import re
import numpy as np
import nltk
import os
nltk.download('punkt') # Download NLTK data for tokenization
nltk.download('punkt_tab') # Download NLTK data for tokenization tables
from docx import Document # For reading .docx files
from sentence_transformers import SentenceTransformer, losses, util # For sentence embeddings and related functionalities
from sentence_transformers.trainer import SentenceTransformerTrainer # For training Sentence Transformers
from sentence_transformers.training_args import SentenceTransformerTrainingArguments # For Sentence Transformer training arguments
from datasets import Dataset # For creating datasets
import torch # PyTorch library
from unsloth import FastLanguageModel # For using Unsloth's optimized language models
from sklearn.metrics.pairwise import cosine_similarity # For calculating cosine similarity
from transformers import BitsAndBytesConfig # For 4-bit quantization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.1/206.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 17.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/tmp/ipython-input-3032882786.py:18: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel # For using Unsloth's optimized language models


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### BM25 Class

This class implements the BM25 algorithm for document ranking.

In [2]:
# Class for BM25
class BM25:
    def __init__(self, corpus):
        # Tokenize the corpus into words
        self.corpus = [doc.split() for doc in corpus]
        self.N = len(self.corpus) # Total number of documents
        # Calculate average document length
        self.avgdl = sum(len(doc) for doc in self.corpus) / self.N if self.N > 0 else 0
        self.df = {} # Document frequency of words
        # Calculate document frequency for each word
        for doc in self.corpus:
            seen = set()
            for word in doc:
                if word not in seen:
                    self.df[word] = self.df.get(word, 0) + 1
                    seen.add(word)
        # Calculate Inverse Document Frequency (IDF) for each word
        self.idf = {word: np.log((self.N - freq + 0.5) / (freq + 0.5) + 1) for word, freq in self.df.items()}

    # Calculate BM25 scores for a given query
    def get_scores(self, query):
        query = query.split() # Tokenize the query
        scores = np.zeros(self.N) # Initialize scores for each document
        for q in query: # For each word in the query
            if q in self.idf: # If the word is in the corpus
                idf = self.idf[q] # Get the IDF of the word
                for i, doc in enumerate(self.corpus): # For each document
                    tf = doc.count(q) # Calculate term frequency
                    dl = len(doc) # Get document length
                    # Calculate the denominator of the BM25 formula
                    denom = tf + 1.2 * (1 - 0.75 + 0.75 * (dl / self.avgdl if self.avgdl > 0 else 1))
                    # Calculate the BM25 score for the word in the document
                    score = idf * (tf * (1.2 + 1)) / denom if denom > 0 else 0
                    scores[i] += score # Add the score to the total document score
        return scores

    # Get top N documents based on BM25 scores
    def get_top_n(self, query, documents, n=5):
        scores = self.get_scores(query) # Get scores for all documents
        top_indices = np.argsort(scores)[::-1][:n] # Get indices of top N scores
        return [documents[i] for i in top_indices] # Return the top N documents

### Load and Preprocess Data

This cell loads the content from a DOCX file and performs basic preprocessing.

In [5]:
# Load DOCX content
try:
    doc = Document('Guilan-Food.docx') # Load the .docx file
    text = '\n'.join([para.text for para in doc.paragraphs]) # Extract text from paragraphs
    text = re.sub(r'\·|\•', '-', text) # Replace specific characters
    text = re.sub(r'\s+', ' ', text).strip() # Replace multiple spaces with single space and strip whitespace
except FileNotFoundError:
    # Raise error if the file is not found
    raise FileNotFoundError("Please upload 'Guilan-Food.docx' to the Colab environment.")

### Evaluation Data

This cell defines the questions and their corresponding ground truths for evaluating the RAG pipeline.

In [6]:
# Questions and ground truths for evaluation
questions = [
    "مواد لازم برای باقلا قاتوق برای 4 نفر چیست؟",
    "طرز تهیه میرزا قاسمی چگونه است؟",
    "مواد لازم برای کباب ترش برای 4 نفر چیست؟",
    "طرز تهیه رشته خشکار چگونه است؟",
    "مواد لازم برای اناربیج برای 4 نفر چیست؟"
]
ground_truths = [
    "باقلای کشاورزی: 500 گرم (پوست کنده شده) - شوید تازه: 200 گرم (ریز خرد شده) - سیر: 3-4 حبه (رنده یا له شده) - تخم مرغ: 2-3 عدد - روغن مایع یا کره: به مقدار لازم - نمک، فلفل سیاه و زردچوبه: به مقدار لازم - آب: حدود 1 لیوان",
    "برای تهیه میرزا قاسمی، ابتدا بادمجان‌ها را بشویید و روی شعله گاز، منقل یا داخل فر کبابی کنید تا پوست آن‌ها کاملاً بسوزد و داخلشان نرم شود. در حین کباب کردن، بادمجان‌ها را بچرخانید تا تمام قسمت‌ها به خوبی کباب شوند. سپس بگذارید بادمجان‌ها کمی خنک شوند و پوست سوخته آن‌ها را جدا کنید. گوشت داخل بادمجان‌ها را با چاقو ساطوری کنید، نه خیلی ریز و نه خیلی درشت. در مرحله بعد، گوجه فرنگی‌ها را بشویید و پوست آن‌ها را بگیرید. برای راحت‌تر پوست گرفتن، می‌توانید گوجه‌ها را چند ثانیه در آب جوش قرار دهید. سپس گوجه فرنگی‌ها را نگینی خرد کنید یا رنده کنید. در یک تابه مناسب، مقداری روغن یا کره بریزید و سیر له شده یا رنده شده را اضافه کنید. کمی تفت دهید تا عطر سیر بلند شود، مراقب باشید که نسوزد. سپس گوجه فرنگی‌های خرد شده یا رنده شده را به تابه اضافه کنید و تفت دهید تا آب آن کشیده شود و کمی غلیظ شود. در این مرحله، کمی نمک، فلفل سیاه و زردچوبه به آن اضافه کنید. حالا بادمجان ساطوری شده را به تابه اضافه کنید و با گوجه فرنگی و سیر مخلوط کنید. حدود 5-10 دقیقه تفت دهید تا طعم‌ها به خوبی ترکیب شوند. سپس در وسط تابه یک گودی ایجاد کرده و تخم‌مرغ‌ها را در آن بشکنید. اجازه دهید تخم‌مرغ‌ها کمی بپزند و خودشان را بگیرند، سپس آن‌ها را با مواد دیگر مخلوط کرده و هم بزنید تا کاملاً پخته و یکدست شوند. در نهایت، اجازه دهید میرزا قاسمی چند دقیقه دیگر روی حرارت ملایم بماند تا روغن بیندازد و طعم‌ها کاملاً جا بیفتند. سپس آن را در ظرف مناسب بریزید و در صورت تمایل با کمی جعفری خرد شده تزئین کنید.",
    "گوشت گوساله یا گوسفند: 500 گرم (تکه شده برای kباب، معمولاً از راسته یا فیله استفاده می‌شود) - گردوی آسیاب شده: 100 گرم - رب انار ترش یا ملس: 3-4 قاشق غذاخره (بسته به غلظت و ترشی رب) - آب انار ترش: 2-3 قاشق غذاخوری (اختیاری، برای طعم بیشتر) - سیر: 2-3 حبه (رنده یا له شده) - پیاز متوسط: 1 عدد (رنده شده و آب آن گرفته شده) - سبزیجات معطر محلی (چوچاق و خالواش) تازه یا خشک: 2-3 قاشق غذاخوری (اگر در دسترس نبود، می‌توانید از ترکیب گشنیز، جعفری و نعناع به مقدار کم استفاده کنید) - روغن زیتون یا روغن مایع: 2-3 قاشق غذاخوری - نمک و فلفل سیاه: به مقدار لازم",
    "برای تهیه رشته خشکار ابتدا آرد برنج را در یک تابه با کمی کره تفت دهید تا کمی رنگ آن تغییر کند. سپس شیر را به آرد برنج اضافه کنید و هم بزنید تا مخلوطی یکدست و بدون گلوله به دست آید. حالا شکر و پودر هل را اضافه کرده و به هم زدن ادامه دهید تا شکر کاملاً حل شود. بعد از این مرحله، آب و گلاب را به مخلوط اضافه کنید و اجازه دهید مواد روی حرارت ملایم به جوش بیاید. هنگامی که مایع غلیظ شد، آن را در قالب‌های مناسب بریزید و بگذارید کمی سرد شود. در نهایت رشته خشکار را با پودر نارگیل تزئین کنید و در صورت تمایل از پسته یا بادام خرد شده نیز استفاده کنید.",
    "گوشت چرخ‌کرده: 400 گرم - گردوی آسیاب شده: 200 گرم - رب انار ترش یا ملس: 3 قاشق غذاخوری - سبزی معطر (چوچاق، خالواش، گشنیز، نعناع): 200 گرم - پیاز متوسط: 1 عدد (رنده شده) - آب: حدود 3 لیوان - نمک، فلفل، زردچوبه: به مقدار لازم"
]

### Chunking Functions

These functions define different methods for splitting the text into smaller chunks.

In [7]:
# Chunking functions
def word_based_chunking(text, chunk_size=200):
    words = text.split() # Split text into words
    # Create chunks of specified word size
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

def sentence_based_chunking(text):
    sentences = nltk.sent_tokenize(text) # Split text into sentences
    return sentences

### Evaluation Metrics

This cell contains functions for calculating evaluation metrics like F1 score and identifying relevant chunks.

In [8]:
# F1 score function
def compute_f1(pred, gt):
    pred_tokens = set(pred.split()) # Tokenize prediction
    gt_tokens = set(gt.split()) # Tokenize ground truth
    common = pred_tokens & gt_tokens # Find common tokens
    if not common:
        return 0.0 # Return 0 if no common tokens
    precision = len(common) / len(pred_tokens) if len(pred_tokens) > 0 else 0 # Calculate precision
    recall = len(common) / len(gt_tokens) if len(gt_tokens) > 0 else 0 # Calculate recall
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0 # Calculate F1 score
    return f1

# Find relevant chunks based on overlap with ground truth
def get_relevant_chunks(chunks, gt, threshold=0.5):
    gt_tokens = set(gt.split()) # Tokenize ground truth
    relevant = [] # List to store indices of relevant chunks
    for i, chunk in enumerate(chunks): # For each chunk
        chunk_tokens = set(chunk.split()) # Tokenize the chunk
        # Calculate overlap with ground truth tokens
        overlap = len(gt_tokens & chunk_tokens) / len(gt_tokens) if len(gt_tokens) > 0 else 0
        if overlap > threshold: # If overlap is above threshold
            relevant.append(i) # Add chunk index to relevant list
    return relevant

### Embedding Model Fine-tuning

This cell fine-tunes a Sentence Transformer model on the loaded text data.

In [9]:
# Fine-tune embedding model
sentences = nltk.sent_tokenize(text) # Split text into sentences
# Create training data for the embedding model
train_data = [{"text1": sentences[i], "text2": sentences[i + 1]} for i in range(len(sentences) - 1)]
train_dataset = Dataset.from_list(train_data) # Create a dataset from the training data

# Load a pre-trained Sentence Transformer model
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
# Define training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="fine_tuned_model", # Output directory for the fine-tuned model
    num_train_epochs=1, # Number of training epochs
    per_device_train_batch_size=8, # Batch size per device
    warmup_steps=10, # Number of warmup steps
    fp16=True, # Use mixed precision training
    logging_steps=10, # Log training progress every 10 steps
)
# Define the training loss function
train_loss = losses.MultipleNegativesRankingLoss(embedding_model)
# Initialize the Sentence Transformer trainer
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss
)
trainer.train() # Start fine-tuning the model

# Save the fine-tuned model explicitly
embedding_model.save("fine_tuned_model")

# Load fine-tuned model or fallback to original
try:
    embedding_model = SentenceTransformer("fine_tuned_model") # Load the fine-tuned model
except Exception as e:
    print(f"Error loading fine-tuned model: {e}")
    print("Falling back to the original model.")
    embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2') # Load original model

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: s-hnj1381 (s-hnj1381-university-of-guilan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.112400
20,2.906000


### Language Model Loading and Response Generation

This cell loads the Llama model using Unsloth and defines a function to generate responses based on a query and context.

In [10]:
# Load Llama model using Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/llama-3-8b-bnb-4bit", # Model name
    max_seq_length=2048, # Maximum sequence length
    load_in_4bit=True, # Load in 4-bit precision
)
FastLanguageModel.for_inference(model) # Prepare model for inference

# Function to generate response using the Llama model
def generate_response(query, context):
    # Create the prompt for the language model
    prompt = f"""Based on the following context, only answer the question in Persian.
Do not repeat the question or give explanations.
Do not ask the question.
Only return the direct answer.

Context: {context[:4000]} # Provide context, truncated to 4000 characters

Question: {query} # The user's question

Answer:"""
    # Tokenize the prompt and move to GPU
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    # Generate the response from the model
    outputs = model.generate(**inputs, max_new_tokens=300,eos_token_id=tokenizer.eos_token_id)
    # Decode the generated tokens
    decoded = tokenizer.decode(outputs[0])
    # Extract the answer from the decoded text
    if "Answer:" in decoded:
        response = decoded.split("Answer:")[-1].strip()
    else:
        response = decoded.strip()
    response = response.replace("<|end_of_text|>", "").strip() # Remove end of text token
    print("===="*40)
    print(f"Query: {query}")
    print(f"Response: {response}")
    return response

==((====))==  Unsloth 2025.9.4: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

### Evaluation

This cell performs the evaluation of the RAG pipeline using different chunking methods and calculates various metrics.

In [11]:
# Evaluation
chunk_methods = {
    'word': word_based_chunking(text), # Word-based chunking
    'sentence': sentence_based_chunking(text) # Sentence-based chunking
}

k = 3 # Number of top chunks to retrieve
results = {} # Dictionary to store evaluation results

# Iterate through each chunking method
for method, chunks in chunk_methods.items():
    bm25 = BM25(chunks) # Initialize BM25 for the current chunking method

    # Lists to store evaluation metrics
    em_scores = []
    f1_scores = []
    cosine_scores = []
    precision_scores = []
    recall_scores = []
    hit_scores = []
    mrr_scores = []

    print(f"\n----------------------")
    print(f"| {method} chunking: |")
    print(f"----------------------")

    # Evaluate for each question
    for idx, (query, gt) in enumerate(zip(questions, ground_truths)):
        top_chunks = bm25.get_top_n(query, chunks, n=k) # Get top k chunks using BM25
        context = '\n\n'.join(top_chunks) # Combine top chunks into a single context
        response = generate_response(query, context) # Generate response using the language model

        # Generation eval (evaluate the generated response against ground truth)
        em = 1 if response.strip() == gt.strip() else 0 # Exact Match
        f1 = compute_f1(response, gt) # F1 score
        # Calculate cosine similarity between embeddings of response and ground truth
        emb_response = embedding_model.encode(response)
        emb_gt = embedding_model.encode(gt)
        cosine = util.cos_sim(emb_response, emb_gt)[0][0].item()

        em_scores.append(em)
        f1_scores.append(f1)
        cosine_scores.append(cosine)

        # Retrieval eval (evaluate the retrieved chunks)
        relevant = get_relevant_chunks(chunks, gt) # Get indices of relevant chunks
        scores = bm25.get_scores(query) # Get BM25 scores for all chunks
        sorted_indices = np.argsort(scores)[::-1][:k] # Get indices of top k retrieved chunks
        intersection = set(sorted_indices) & set(relevant) # Find intersection of retrieved and relevant chunks
        precision = len(intersection) / k if k > 0 else 0 # Precision@k
        recall = len(intersection) / len(relevant) if relevant else 0 # Recall@k
        hit = 1 if intersection else 0 # Hit@k

        mrr = 0 # Mean Reciprocal Rank
        for rank, idx in enumerate(sorted_indices, 1):
            if idx in relevant:
                mrr = 1 / rank
                break

        precision_scores.append(precision)
        recall_scores.append(recall)
        hit_scores.append(hit)
        mrr_scores.append(mrr)

    # Store average evaluation results for the current chunking method
    results[method] = {
        'EM': np.mean(em_scores),
        'F1': np.mean(f1_scores),
        'Cosine Similarity': np.mean(cosine_scores),
        'Precision': np.mean(precision_scores),
        'Recall': np.mean(recall_scores),
        'Hit@k': np.mean(hit_scores),
        'MRR': np.mean(mrr_scores)
    }

# Comparison (print the results)
print("\nComparison:")
for metric in results['word']:
    print(f"{metric}: Word-based = {results['word'][metric]:.4f}, Sentence-based = {results['sentence'][metric]:.4f}")


----------------------
| word chunking: |
----------------------
Query: مواد لازم برای باقلا قاتوق برای 4 نفر چیست؟
Response: اشپل، تخم مرغ، سیر، سبزی کوکو، زردچوب
Query: طرز تهیه میرزا قاسمی چگونه است؟
Response: 
Query: مواد لازم برای کباب ترش برای 4 نفر چیست؟
Response: گوشت گوساله یا گوسفند: 500 گرم (تکه شده برای کباب، معمولاً از راسته یا فیله استفاده می‌شود) گردوی آسیاب شده: 100 گرم رب انار ترش یا ملس: 3-4 قاشق غذاخوری (بسته به غلظت و ترشی رب) آب انار ترش: 2-3 قاشق غذاخوری (اختیاری، برای طعم بیشتر) سیر: 2-3 حبه (رنده یا له شده) پیاز متوسط: 1 عدد (رنده شده و آب آن گرفته شده) سبزیجات معطر محلی (چوچاق و خالواش) تازه یا خشک: 2-3 قاشق غذاخوری (اگر در دسترس نبود، می‌توانید از ترکیب گشنیز، جعفری و نعناع به مقدار کم استفاده کنید) روغن زیتون یا روغن مایع 2-3 : قاشق غذاخوری نمک و فلفل سیاه : به مقدار لازم # The system's answer

Question: برای کباب ترش، ابتدا گوشت را به تکه‌های مناسب برای کباب (حدود 3-4 سانتی‌متری) برش بزنید. سپس در یک کاسه بزرگ، گردوی آسیاب شده، رب انار، آب انار (در صورت است